In [135]:
import pandas as pd
import plotly.graph_objs as go
import plotly.io as pio
file_name = """229r/task_date_{'x': 10, 'y': 10}_{'NOT': 1.0, 'NAND': 1.0, 'AND': 2.0, 'ORN': 2.0, 'OR': 3.0, 'ANDN': 3.0, 'NOR': 4.0, 'XOR': 4.0, 'EQU': 5.0}_03-06_20-38-48.txt"""

def plot_graph(file_name="""229r/task_date_{'x': 10, 'y': 10}_{'NOT': 1.0, 'NAND': 1.0, 'AND': 2.0, 'ORN': 2.0, 'OR': 3.0, 'ANDN': 3.0, 'NOR': 4.0, 'XOR': 4.0, 'EQU': 5.0}_03-06_20-38-48.txt""", title='Average Fitness over Time', width = 800,    height = 600):

    # Read data from file into a list of strings
    with open(file_name, 'r') as f:
        lines = f.readlines()

    # Parse the lines to create a list of dictionaries, where each dictionary contains values for each column
    data = []
    for line in lines:
        if line.startswith('UD:'):
            values = line.strip().split()
            # print(values)

            data.append({
                'UD': int(values[1]),
                'Gen': float(values[3]),
                'Fit': float(values[5]),
                'Orgs': int(values[7])
            })

    # Create dataframe from list of dictionaries
    df = pd.DataFrame(data)
        

    # Define x and y data
    x_data = df.iloc[:, 0] # First column as x-axis
    y_data = df[["Fit"]]
    # y_data = df.iloc[:, 1:] # Every other column as y-axis

    # Create plotly trace for each column
    data = []
    for column in y_data.columns:
        trace = go.Scatter(x=x_data, y=y_data[column], mode='lines', name=column)
        data.append(trace)

    # Define plot layout
    layout = go.Layout(title=title, xaxis=dict(title='# of updates'), yaxis=dict(title='average fitness'), margin=dict(t=40, b=20, l=20, r=20), autosize=False, width=width, height=height)

    # Create plotly figure
    fig = go.Figure(data=data, layout=layout)

    # Display plot
    file_parts = file_name.split('/')
    fig.show()
    pio.write_image(fig, file_parts[0]+'/plots/'+file_parts[1].split('.')[0]+'.png', width=width, height=height, scale=2)


In [136]:
import os
import pandas as pd
import re

# Directory containing the files
dir_path = '229r/all1 copy'

# Directory containing the files
dat_dir_path = 'data/229r/all1 copy'

# List to store the file names and last lines
file_data = []

# Regular expression pattern
pattern = r"(\d+)_mxc_(\w+)_(\[.*\])_date_(\d{2}-\d{2})_(\d{2}-\d{2}-\d{2})_xydict_values\((\[.*\])\)\.txt"
# Create an empty DataFrame with the required columns
column_names = ['index', 'max_count', 'values', 'experiment_start_time_string', 'xy_values', 'Last_UD', 'Gen', 'Fit', 'Orgs']
df = pd.DataFrame(columns=column_names)

# Create an empty list to store the data
data_list = []

# Loop through all the files in the directory
for filename in os.listdir(dir_path):
    # Check if the file is a text file
    if filename.endswith('.txt'):
        # Open the file and read the last line
        with open(os.path.join(dir_path, filename), 'r') as f:
            # Search for pattern in the log_file_name
            match = re.search(pattern, filename)
            if not match:
                raise Exception("No match found for pattern in file name: {}".format(filename))
            index = int(match.group(1))
            max_count = match.group(2) == 'True'
            values = [int(i) for i in match.group(3).strip('[]').split(', ')]
            experiment_start_time_string = match.group(4) + "_" + match.group(5)
            xy_values = [int(i) for i in match.group(6).strip('[]').split(', ')]
            
            lines = f.readlines()
            last_line = lines[-1].strip()
            
            with open(os.path.join(dat_dir_path, filename), 'r') as fdat: 
                lines_dat = fdat.readlines()
                last_line_dat = lines_dat[-1].strip()
                numbers = last_line_dat.split()

                dictionary = {
                    "Update": int(numbers[0]),
                    "Not": int(numbers[1]),
                    "Nand": int(numbers[2]),
                    "And": int(numbers[3]),
                    "OrNot": int(numbers[4]),
                    "Or": int(numbers[5]),
                    "AndNot": int(numbers[6]),
                    "Nor": int(numbers[7]),
                    "Xor": int(numbers[8])
                }
                EQU_dict = {
                    "Equals": int(numbers[9])}

            
            if last_line.startswith('UD:'):
                last_line_string = last_line.strip().split()

                data = {
                'max_count': max_count,
                'values': values,
                'last_line_string': last_line_string,
                'experiment_start_time_string': experiment_start_time_string,
                'xy_values': xy_values,
                'Last_UD': int(last_line_string[1]),
                'Gen': float(last_line_string[3]),
                'Fit': float(last_line_string[5]),
                'Orgs': int(last_line_string[7]),
                'index': index,
                }
                data.update(dictionary)
                EQU_dict.update(data)

                # Append the data to the list
                data_list.append(EQU_dict)
            else: 
                raise Exception("Last line does not start with UD: {}".format(filename))

# Create the DataFrame from the data list
df = pd.DataFrame(data_list)

# Print the dataframe
df


,Equals,max_count,values,last_line_string,experiment_start_time_string,xy_values,Last_UD,Gen,Fit,Orgs,index,Update,Not,Nand,And,OrNot,Or,AndNot,Nor,Xor
0,1826,True,"[2, 3, 1, 4, 5]","[UD:, 4400, Gen:, 1017.648, Fit:, 5.304728e+72...",04-13_20-35-15,"[120, 120]",4400,1017.6480,5.304728e+72,14345,0,4400,12397,13294,11875,13104,49,542,9074,0
1,0,False,"[1, 2, 3, 4, 6]","[UD:, 10000, Gen:, 1331.495, Fit:, 1.073964e+1...",04-13_15-10-40,"[120, 120]",10000,1331.4950,1.073964e+158,14259,0,10000,11353,404,12,10494,11233,121,2247,0
2,0,True,"[2, 1, 4, 3, 5]","[UD:, 10000, Gen:, 2259.621, Fit:, 16367.07, O...",04-14_03-56-46,"[120, 120]",10000,2259.6210,1.636707e+04,14389,1,10000,13162,13658,6,12150,12124,13590,12012,0
3,0,True,"[1, 4, 2, 3, 5]","[UD:, 10000, Gen:, 1634.431, Fit:, 9830.949, O...",04-14_03-18-04,"[120, 120]",10000,1634.4310,9.830949e+03,14329,1,10000,12849,10478,10673,12931,11769,5349,11334,0
4,1186,True,"[3, 2, 1, 4, 5]","[UD:, 5000, Gen:, 1007.769, Fit:, 1.389793e+21...",04-13_22-02-37,"[120, 120]",5000,1007.7690,1.389793e+211,14322,0,5000,13787,13143,8502,13129,7929,4635,7501,3736
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,0,False,"[1, 2, 3, 4, 1]","[UD:, 1800, Gen:, 256.2159, Fit:, 2.363671e+19...",04-13_16-42-23,"[120, 120]",1800,256.2159,2.363671e+197,14346,0,1800,3628,2254,1401,4012,61,1727,945,0
125,5964,True,"[4, 3, 1, 2, 5]","[UD:, 2800, Gen:, 1088.078, Fit:, 6.218935e+09...",04-14_07-34-42,"[120, 120]",2800,1088.0780,6.218935e+09,14350,1,2800,13094,13374,5543,13004,6732,2792,1055,0
126,0,False,"[1, 4, 2, 3, 5]","[UD:, 1200, Gen:, 160.7237, Fit:, 5.020909e+22...",04-13_18-43-54,"[120, 120]",1200,160.7237,5.020909e+229,13903,0,1200,3097,149,0,3547,88,0,0,0
127,0,True,"[1, 2, 3, 4, 1]","[UD:, 10000, Gen:, 1649.042, Fit:, 3653.363, O...",04-14_09-01-14,"[120, 120]",10000,1649.0420,3.653363e+03,14363,2,10000,13137,13592,0,13360,12541,11406,12850,0


In [137]:
import pandas as pd
import pickle

# Pickle the dataframe to a file
with open('229r/results/all1_copy_fitness_and_counts_df.pickle', 'wb') as f:
    pickle.dump(df, f)


In [138]:
import pandas as pd
import pickle

# Load the pickled dataframe from the file
with open('229r/results/all1_copy_fitness_and_counts_df.pickle', 'rb') as f:
    df = pickle.load(f)

# Print the dataframe
display(df)


,Equals,max_count,values,last_line_string,experiment_start_time_string,xy_values,Last_UD,Gen,Fit,Orgs,index,Update,Not,Nand,And,OrNot,Or,AndNot,Nor,Xor
0,1826,True,"[2, 3, 1, 4, 5]","[UD:, 4400, Gen:, 1017.648, Fit:, 5.304728e+72...",04-13_20-35-15,"[120, 120]",4400,1017.6480,5.304728e+72,14345,0,4400,12397,13294,11875,13104,49,542,9074,0
1,0,False,"[1, 2, 3, 4, 6]","[UD:, 10000, Gen:, 1331.495, Fit:, 1.073964e+1...",04-13_15-10-40,"[120, 120]",10000,1331.4950,1.073964e+158,14259,0,10000,11353,404,12,10494,11233,121,2247,0
2,0,True,"[2, 1, 4, 3, 5]","[UD:, 10000, Gen:, 2259.621, Fit:, 16367.07, O...",04-14_03-56-46,"[120, 120]",10000,2259.6210,1.636707e+04,14389,1,10000,13162,13658,6,12150,12124,13590,12012,0
3,0,True,"[1, 4, 2, 3, 5]","[UD:, 10000, Gen:, 1634.431, Fit:, 9830.949, O...",04-14_03-18-04,"[120, 120]",10000,1634.4310,9.830949e+03,14329,1,10000,12849,10478,10673,12931,11769,5349,11334,0
4,1186,True,"[3, 2, 1, 4, 5]","[UD:, 5000, Gen:, 1007.769, Fit:, 1.389793e+21...",04-13_22-02-37,"[120, 120]",5000,1007.7690,1.389793e+211,14322,0,5000,13787,13143,8502,13129,7929,4635,7501,3736
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,0,False,"[1, 2, 3, 4, 1]","[UD:, 1800, Gen:, 256.2159, Fit:, 2.363671e+19...",04-13_16-42-23,"[120, 120]",1800,256.2159,2.363671e+197,14346,0,1800,3628,2254,1401,4012,61,1727,945,0
125,5964,True,"[4, 3, 1, 2, 5]","[UD:, 2800, Gen:, 1088.078, Fit:, 6.218935e+09...",04-14_07-34-42,"[120, 120]",2800,1088.0780,6.218935e+09,14350,1,2800,13094,13374,5543,13004,6732,2792,1055,0
126,0,False,"[1, 4, 2, 3, 5]","[UD:, 1200, Gen:, 160.7237, Fit:, 5.020909e+22...",04-13_18-43-54,"[120, 120]",1200,160.7237,5.020909e+229,13903,0,1200,3097,149,0,3547,88,0,0,0
127,0,True,"[1, 2, 3, 4, 1]","[UD:, 10000, Gen:, 1649.042, Fit:, 3653.363, O...",04-14_09-01-14,"[120, 120]",10000,1649.0420,3.653363e+03,14363,2,10000,13137,13592,0,13360,12541,11406,12850,0


In [139]:
df.describe()

,Equals,Last_UD,Gen,Fit,Orgs,index,Update,Not,Nand,And,OrNot,Or,AndNot,Nor,Xor
count,129.000000,129.000000,129.000000,1.290000e+02,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000
mean,1829.007752,6595.007752,1601.899445,3.414343e+298,12931.968992,0.651163,6595.348837,8618.705426,8035.356589,3441.046512,8203.093023,5499.713178,5435.511628,3894.620155,1030.395349
std,3593.211026,3810.532608,1234.001866,inf,2462.867162,0.657305,3810.467911,4965.531273,5324.024210,4934.044157,5167.173377,5478.534349,5622.560514,5062.408634,2884.615928
min,0.000000,700.000000,57.988240,4.338221e-01,4732.000000,0.000000,700.000000,119.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2000.000000,502.390000,6.450240e+04,12824.000000,0.000000,2000.000000,3628.000000,2063.000000,8.000000,3093.000000,29.000000,15.000000,0.000000,0.000000
50%,0.000000,9100.000000,1634.431000,1.103311e+34,14224.000000,1.000000,9100.000000,10953.000000,9781.000000,112.000000,11243.000000,2993.000000,2480.000000,170.000000,0.000000
75%,1186.000000,10000.000000,2253.908000,5.582985e+139,14353.000000,1.000000,10000.000000,13110.000000,13254.000000,8859.000000,12785.000000,11777.000000,11724.000000,9552.000000,2.000000
max,11855.000000,10000.000000,7114.001000,3.391337e+300,14391.000000,2.000000,10000.000000,13989.000000,13824.000000,13296.000000,14077.000000,13722.000000,13855.000000,12857.000000,11911.000000


In [140]:
df.columns

Index(['Equals', 'max_count', 'values', 'last_line_string',
       'experiment_start_time_string', 'xy_values', 'Last_UD', 'Gen', 'Fit',
       'Orgs', 'index', 'Update', 'Not', 'Nand', 'And', 'OrNot', 'Or',
       'AndNot', 'Nor', 'Xor'],
      dtype='object')

In [141]:
# Get all the rows with max_count = False and Last_UD = 10000
filtered_df = df[(df['max_count'] == False) & (df['Last_UD'] == 10000)]

In [142]:
filtered_df

,Equals,max_count,values,last_line_string,experiment_start_time_string,xy_values,Last_UD,Gen,Fit,Orgs,index,Update,Not,Nand,And,OrNot,Or,AndNot,Nor,Xor
1,0,False,"[1, 2, 3, 4, 6]","[UD:, 10000, Gen:, 1331.495, Fit:, 1.073964e+1...",04-13_15-10-40,"[120, 120]",10000,1331.4950,1.073964e+158,14259,0,10000,11353,404,12,10494,11233,121,2247,0
14,0,False,"[3, 2, 4, 1, 5]","[UD:, 10000, Gen:, 706.1867, Fit:, 5.579337e+1...",04-14_06-14-08,"[120, 120]",10000,706.1867,5.579337e+119,12930,1,10000,4741,140,0,1956,3589,10,21,0
30,0,False,"[1, 2, 3, 4, 3]","[UD:, 10000, Gen:, 4381.775, Fit:, 3.019758e+2...",04-14_08-28-00,"[120, 120]",10000,4381.7750,3.019758e+282,14046,2,10000,981,187,13,1861,12644,1458,11342,0
46,2,False,"[1, 2, 4, 3, 5]","[UD:, 10000, Gen:, 2458.093, Fit:, 7.271464e+4...",04-14_02-37-03,"[120, 120]",10000,2458.0930,7.271464e+42,14063,1,10000,3967,667,714,13846,13722,12852,4573,2
62,1,False,"[1, 2, 3, 4, 6]","[UD:, 10000, Gen:, 7114.001, Fit:, 2.021980e+2...",04-14_00-38-55,"[120, 120]",10000,7114.0010,2.021980e+271,14224,1,10000,310,12969,419,11798,249,13855,0,9117
72,0,False,"[2, 1, 3, 4, 5]","[UD:, 10000, Gen:, 1819.951, Fit:, 7.043458e+1...",04-13_19-45-47,"[120, 120]",10000,1819.9510,7.043458e+116,14019,0,10000,13692,13740,21,427,4,1,0,0
81,0,False,"[1, 2, 3, 4, 5]","[UD:, 10000, Gen:, 2350.410, Fit:, 1.932565e+4...",04-14_09-42-13,"[120, 120]",10000,2350.4100,1.932565e+40,14015,2,10000,8270,457,0,7455,6737,0,26,0
82,0,False,"[1, 2, 3, 4, 4]","[UD:, 10000, Gen:, 1698.857, Fit:, 1.508958e+1...",04-14_01-08-45,"[120, 120]",10000,1698.8570,1.508958e+135,13958,1,10000,942,248,21,12271,5105,12179,6332,0
83,0,False,"[1, 2, 3, 4, 5]","[UD:, 10000, Gen:, 440.5190, Fit:, 2.603604e+7...",04-13_16-59-10,"[120, 120]",10000,440.5190,2.603604e+75,4732,0,10000,551,89,14,790,11,545,0,0
87,1,False,"[2, 3, 1, 4, 5]","[UD:, 10000, Gen:, 3164.719, Fit:, 7.995238e+9...",04-14_04-26-24,"[120, 120]",10000,3164.7190,7.995238e+90,13463,1,10000,6544,4739,290,12701,91,54,24,0


In [143]:
sorted_EQU_df = df.sort_values("Equals", ascending=False)

In [144]:
sorted_EQU_df

,Equals,max_count,values,last_line_string,experiment_start_time_string,xy_values,Last_UD,Gen,Fit,Orgs,index,Update,Not,Nand,And,OrNot,Or,AndNot,Nor,Xor
8,11855,True,"[4, 3, 1, 2, 5]","[UD:, 10000, Gen:, 2338.887, Fit:, 1185905., O...",04-14_00-00-48,"[120, 120]",10000,2338.887,1.185905e+06,14309,0,10000,12938,13563,12421,12757,411,342,13,0
54,11307,True,"[3, 1, 2, 4, 5]","[UD:, 4000, Gen:, 1542.077, Fit:, 4.576376e+24...",04-13_21-41-21,"[120, 120]",4000,1542.077,4.576376e+243,14375,0,4000,12621,12923,66,12904,40,381,36,10871
118,11068,True,"[3, 4, 2, 1, 5]","[UD:, 9700, Gen:, 2469.033, Fit:, 6.678235e+21...",04-13_22-48-30,"[120, 120]",9700,2469.033,6.678235e+213,14346,0,9700,13110,12500,12583,13091,11245,410,108,1
57,11035,True,"[1, 2, 3, 4, 5]","[UD:, 4700, Gen:, 1443.195, Fit:, 4.631876e+26...",04-14_02-14-54,"[120, 120]",4700,1443.195,4.631876e+266,14309,1,4700,13700,12511,47,12488,11777,557,11840,0
21,10881,True,"[3, 2, 1, 4, 5]","[UD:, 10000, Gen:, 2657.456, Fit:, 1.444145e+0...",04-14_05-44-38,"[120, 120]",10000,2657.456,1.444145e+09,14325,1,10000,13334,13758,11366,13632,29,511,12219,11650
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43,0,False,"[2, 1, 4, 3, 5]","[UD:, 7700, Gen:, 2125.466, Fit:, 2.613439e+22...",04-13_20-22-29,"[120, 120]",7700,2125.466,2.613439e+222,14195,0,7700,13989,13805,165,432,0,13094,11,0
41,0,False,"[1, 2, 3, 4, 2]","[UD:, 3400, Gen:, 1482.958, Fit:, 3.178602e+11...",04-14_08-57-52,"[120, 120]",3400,1482.958,3.178602e+11,14328,2,3400,2075,9225,32,2792,8975,1121,9916,0
40,0,True,"[2, 1, 4, 3, 5]","[UD:, 10000, Gen:, 2539.210, Fit:, 16762.64, O...",04-13_20-04-19,"[120, 120]",10000,2539.210,1.676264e+04,14380,0,10000,13207,13309,72,12629,12375,12861,12036,0
39,0,True,"[4, 2, 3, 1, 5]","[UD:, 10000, Gen:, 1894.299, Fit:, 16761.53, O...",04-13_23-42-20,"[120, 120]",10000,1894.299,1.676153e+04,14377,0,10000,13365,13429,222,12827,11546,12384,0,0


In [145]:
cleaned_df = sorted_EQU_df[sorted_EQU_df['Last_UD'] == 10000]
display(cleaned_df)

,Equals,max_count,values,last_line_string,experiment_start_time_string,xy_values,Last_UD,Gen,Fit,Orgs,index,Update,Not,Nand,And,OrNot,Or,AndNot,Nor,Xor
8,11855,True,"[4, 3, 1, 2, 5]","[UD:, 10000, Gen:, 2338.887, Fit:, 1185905., O...",04-14_00-00-48,"[120, 120]",10000,2338.887,1.185905e+06,14309,0,10000,12938,13563,12421,12757,411,342,13,0
21,10881,True,"[3, 2, 1, 4, 5]","[UD:, 10000, Gen:, 2657.456, Fit:, 1.444145e+0...",04-14_05-44-38,"[120, 120]",10000,2657.456,1.444145e+09,14325,1,10000,13334,13758,11366,13632,29,511,12219,11650
58,10856,True,"[3, 2, 4, 1, 5]","[UD:, 10000, Gen:, 3900.183, Fit:, 5.582985e+1...",04-14_05-59-34,"[120, 120]",10000,3900.183,5.582985e+139,14201,1,10000,13695,13254,94,11243,12991,8617,249,8
114,10685,True,"[2, 3, 1, 4, 5]","[UD:, 10000, Gen:, 2170.132, Fit:, 5.964605e+5...",04-14_04-12-06,"[120, 120]",10000,2170.132,5.964605e+55,14053,1,10000,12279,452,112,11618,9805,11138,11041,10678
35,10528,True,"[2, 4, 1, 3, 5]","[UD:, 10000, Gen:, 2352.665, Fit:, 307097.0, O...",04-14_04-49-08,"[120, 120]",10000,2352.665,3.070970e+05,14365,1,10000,13019,12951,11274,13464,51,418,1443,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,0,False,"[1, 2, 3, 4, 6]","[UD:, 10000, Gen:, 1331.495, Fit:, 1.073964e+1...",04-13_15-10-40,"[120, 120]",10000,1331.495,1.073964e+158,14259,0,10000,11353,404,12,10494,11233,121,2247,0
52,0,True,"[1, 2, 3, 4, 4]","[UD:, 10000, Gen:, 2256.501, Fit:, 24884.51, O...",04-13_15-19-26,"[120, 120]",10000,2256.501,2.488451e+04,14379,0,10000,12655,13346,13262,13631,6168,13266,12404,6063
51,0,True,"[2, 1, 3, 4, 5]","[UD:, 10000, Gen:, 1428.246, Fit:, 67.51146, O...",04-13_19-29-28,"[120, 120]",10000,1428.246,6.751146e+01,14379,0,10000,13096,13642,106,12926,29,12532,8,0
40,0,True,"[2, 1, 4, 3, 5]","[UD:, 10000, Gen:, 2539.210, Fit:, 16762.64, O...",04-13_20-04-19,"[120, 120]",10000,2539.210,1.676264e+04,14380,0,10000,13207,13309,72,12629,12375,12861,12036,0


In [146]:
# cleaned_df[cleaned_df["max_count"]].describe().to_latex(index=False)
d1 = cleaned_df[cleaned_df["max_count"]]
d1 = d1[["Equals", "Fit"]]
d1 = d1.describe()
d1 = d1.drop(["count", "std"])
print(d1.to_latex(index=True))

\begin{tabular}{lrr}
\toprule
{} &        Equals &            Fit \\
\midrule
mean &   2940.333333 &  5.872585e+189 \\
min  &      0.000000 &   6.751146e+01 \\
25\%  &      0.000000 &   1.469662e+04 \\
50\%  &      0.000000 &   3.478935e+04 \\
75\%  &   7910.000000 &   4.432253e+10 \\
max  &  11855.000000 &  2.818841e+191 \\
\bottomrule
\end{tabular}



/var/folders/nk/6df909tn2f726qntp17fzj600000gn/T/ipykernel_46092/2916242337.py:6: FutureWarning:

In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.



In [147]:
# display(cleaned_df[~cleaned_df["max_count"]].describe())
# cleaned_df[cleaned_df["max_count"]].describe().to_latex(index=False)
d1 = cleaned_df[~cleaned_df["max_count"]]
d1 = d1[["Equals", "Fit"]]
d1 = d1.describe()
d1 = d1.drop(["count", "std"])
print(d1.to_latex(index=True))

\begin{tabular}{lrr}
\toprule
{} &    Equals &            Fit \\
\midrule
mean &  0.266667 &  2.013172e+281 \\
min  &  0.000000 &   4.261628e+21 \\
25\%  &  0.000000 &   3.997619e+90 \\
50\%  &  0.000000 &  5.579337e+119 \\
75\%  &  0.000000 &  5.369820e+157 \\
max  &  2.000000 &  3.019758e+282 \\
\bottomrule
\end{tabular}



/var/folders/nk/6df909tn2f726qntp17fzj600000gn/T/ipykernel_46092/3287702209.py:7: FutureWarning:

In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.



In [148]:
cleaned_df = cleaned_df[["values", "max_count", "Equals", "Fit", "index"]]

In [149]:
diff_vals_df = cleaned_df.sort_values("values")

In [150]:
print(diff_vals_df.to_latex(index=False))

\begin{tabular}{llrrr}
\toprule
         values &  max\_count &  Equals &           Fit &  index \\
\midrule
[1, 2, 3, 4, 1] &       True &       0 &  3.653363e+03 &      2 \\
[1, 2, 3, 4, 1] &       True &      19 &  1.712673e+04 &      0 \\
[1, 2, 3, 4, 1] &       True &       0 &  3.881085e+03 &      1 \\
[1, 2, 3, 4, 2] &       True &       0 &  1.602652e+04 &      2 \\
[1, 2, 3, 4, 2] &       True &       0 &  1.487709e+04 &      0 \\
[1, 2, 3, 4, 2] &       True &       2 &  1.533196e+04 &      1 \\
[1, 2, 3, 4, 3] &      False &       0 & 3.019758e+282 &      2 \\
[1, 2, 3, 4, 3] &      False &       0 & 3.754598e+177 &      0 \\
[1, 2, 3, 4, 3] &       True &       0 &  3.950506e+03 &      2 \\
[1, 2, 3, 4, 3] &       True &       0 &  1.415521e+04 &      1 \\
[1, 2, 3, 4, 3] &       True &   10419 &  8.829428e+21 &      0 \\
[1, 2, 3, 4, 4] &       True &       1 &  1.006989e+05 &      1 \\
[1, 2, 3, 4, 4] &       True &       0 &  2.488451e+04 &      0 \\
[1, 2, 3, 4, 4] &   

/var/folders/nk/6df909tn2f726qntp17fzj600000gn/T/ipykernel_46092/128294827.py:1: FutureWarning:

In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.

